# Modeling Crop Yield
## Python modules

In [ ]:
import warnings
import time
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import geopandas

import pyarrow
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from scipy.stats import spearmanr
from scipy.linalg import LinAlgWarning
from scipy.stats import pearsonr

import math
import seaborn as sns

## Parameters

In [ ]:
# satellite = "landsat-8-c2-l2"
satellite = "sentinel-2-l2a"

# bands = "2-3-4"
bands = "2-3-4-8"
# bands = "1-2-3-4-5-6-7"

country_code = "ZMB"

# points = "4"
points = "15"
# points = "24"
# points = "42"

# year_start = 2013 # Landsat
year_start = 2015 # Sentinel
year_end = 2018

num_features = "1000"

feature_file_name = (f'{satellite}_bands-{bands}_{country_code}_{points}k-points_{num_features}-features')
weight_file_name = (f'{country_code}_crop_weights_{points}k-points')

if points == "4":
    marker_sz = 60
elif points == "15":
    marker_sz = 15
elif points == "24":
    marker_sz = 10
else:
    marker_sz = 8

## Administrative boundaries 

In [ ]:
country_shp = geopandas.read_file(f'/capstone/cropmosaiks/data/boundaries/gadm36_{country_code}_2.shp')
country_shp = country_shp.rename(columns = {'NAME_2': 'district'})[['district', 'geometry']]
country_shp.district = country_shp.district.replace("MPongwe", 'Mpongwe', regex=True)
country_districts = country_shp.district.sort_values().unique().tolist()
country_shp = country_shp.set_index('district')
country_shp.shape
# country_shp.plot(column = 'district',linewidth = 1, edgecolor = 'black' )
# country_shp

## Crops

In [ ]:
crop_df = pd.read_csv('/capstone/cropmosaiks/data/crops/cfs_maize_districts_zambia_2009_2018.csv')
crop_df.district = crop_df.district.replace(
    {"Itezhi-tezhi": 'Itezhi-Tezhi',
     "Kapiri-Mposhi": 'Kapiri Mposhi',
     "Shang'ombo": 'Shangombo',
     "Chienge": 'Chiengi'
    }, regex=True)
crop_districts = crop_df.district.sort_values().unique().tolist()
crop_df = crop_df[['district', 'year', 'yield_mt']]
ln = len(crop_df[crop_df.year == 2016].district)
crop_df = crop_df.set_index('district')
ln
# crop_df

In [ ]:
list(set(crop_districts) - set(country_districts))

In [ ]:
list(set(country_districts) - set(crop_districts))

In [ ]:
country_crop = geopandas.GeoDataFrame(crop_df.join(country_shp), crs = country_shp.crs)

## Weights

In [ ]:
weights = pd.read_feather(f"/capstone/cropmosaiks/data/weights/{weight_file_name}_2018.feather")
# weights

In [ ]:
weights_gdf = geopandas.GeoDataFrame(
    weights, 
    geometry = geopandas.points_from_xy(x = weights.lon, y = weights.lat), 
    crs='EPSG:4326'
)
weights_gdf.plot(figsize = (10,10),
                 cmap = 'inferno',
                 markersize = marker_sz,
                 alpha = .9,
                 column = 'crop_perc')
# plt.axis('off')

In [ ]:
weights.crop_perc = weights.crop_perc.fillna(0)
# #weights.crop_perc = weights.crop_perc + 0.0001

## Features

In [ ]:
features_raw = geopandas.GeoDataFrame()

for yr in range(year_start, year_end + 1):
    print(f"Opening: {feature_file_name}_{yr}.feather")
    features_x = pd.read_feather(f"/capstone/cropmosaiks/data/features/{feature_file_name}_{yr}.feather")
    
    if (yr == 2013) & (satellite == "landsat-8-c2-l2"):
        features_x = features_x[features_x.month > 9]
    elif (yr == 2015) & (satellite == "sentinel-2-l2a"):
        features_x = features_x[features_x.month > 9]
    else:
        pass
    
    # concatenate the feather files together, axis = 0 specifies to stack rows (rather than adding columns)
    features_raw = pd.concat([features_raw, features_x], axis=0)
    
    print("feature.shape", features_raw.shape)
    print("Appending:", yr)
    print("")

In [ ]:
features = features_raw.copy()

In [ ]:
features['year'] = np.where(
    features['month'].isin([10, 11, 12]),
    features['year'] + 1, 
    features['year'])

features = features[features['year'] <= year_end]

features.sort_values(['year', 'month'], inplace=True)

### Filter month range?

In [ ]:
month_range = [
#     1,
#     2,
#     3,
    4,
    5,
    6,
    7,
    8,
    9,
#     10,
#     11,
#     12
]
features = features[features.month.isin(month_range)]

### Pivot wider
Here we pivot the data from long format to wide by indexing on 'lon','lat', "year", 'month' and using the unstack function. We then map column names based on the month index and the associated features so month '01' is appended to each feature for that month making 0_01, 1_01 etc. 

In [ ]:
features = features.set_index(['lon','lat', "year", 'month']).unstack()
features.columns = features.columns.map(lambda x: '{}_{}'.format(*x))

### Replace "inf" values with NaN

In [ ]:
features.replace([np.inf, -np.inf], np.nan, inplace=True)

### Impute missing values

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

impute_simple = True
# impute_simple = False

# impute_iterative = True
impute_iterative = False

if impute_simple:
    imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    imputer.fit_transform(features)
    features[:] = imputer.transform(features)
    features = features.reset_index()
elif impute_iterative:
    imputer = IterativeImputer(max_iter=2, random_state=0)
    imputer.fit_transform(features)
    features[:] = imputer.transform(features)
    features = features.reset_index()
else:
    features = features.fillna(0).reset_index()
features_copy = features.copy()

### Attach crop weights
Attach weight to each point (% area cropped of surrounding 1 km^2).

In [ ]:
features = features.join(weights.set_index(['lon', 'lat']), on = ['lon', 'lat'])
features = features.drop(["geometry"], axis = 1)
# features

### Mask un-croppped regions?

In [ ]:
features = features[features.crop_perc > 0]
# features

### Make "features" a `GeoDataFrame`

In [ ]:
features = geopandas.GeoDataFrame(
    features, 
    geometry = geopandas.points_from_xy(x = features.lon, y = features.lat), 
    crs='EPSG:4326'
)

### Plot any single feature

In [ ]:
# mn = 9
# yr = 2017
# feature = 999

# features[features.year == yr].plot(
#     column = f"{feature}_{mn}",
#     figsize = (10,10),
#     marker='H',
#     # legend = True,
#     markersize = marker_sz,
# )

### Drop 'lat' and 'lon' columns

In [ ]:
# Drop lon and lat now that they are in geometry column
features = features.drop(['lon', 'lat'], axis = 1)

### Join features to country geometry

In [ ]:
features = features.sjoin(country_shp, how = 'left', predicate = 'within')
# features

In [ ]:
# na = features[adm_features.isna().any(axis = 1)]
# na.plot(figsize = (10,10), markersize = 10)

### Fix column names and drop geometry

In [ ]:
features = (
    features
    .dropna(subset=['index_right'])
    .rename(columns = {"index_right": "district",})
    .reset_index(drop = True)
)
features_gdf = features.copy()
features = features.drop(['geometry'], axis = 1)
# features

### Summarise to administrative boundary level
Currently un-weighted by cropped area

In [ ]:
features_summary = features.groupby(['district',"year"], as_index = False).mean()
# features_summary

In [ ]:
# # This is where we are trying to do a weighted average

# features_summary = features.groupby(['district', 'yield_mt' ,"year"], as_index = False
#                                    ).apply(lambda x: np.average(x['0'], weights=x['crop_perc'], axis = 0))
# features_summary

### Join crop data

In [ ]:
crop_df_x = crop_df[crop_df.year >= year_start + 1]
crop_df_x = crop_df_x[~crop_df_x.index.isin(['Mafinga', 'Ikelenge'])]
crop_df_x.reset_index(inplace=True)
# crop_df_x

In [ ]:
features_summary = (features_summary
                    .set_index(["district", "year"])
                    .join(other = crop_df_x.set_index(["district", "year"]))
                    .reset_index())
# features_summary

## Model

In [ ]:
model_year = features_summary[features_summary.year.isin([
#    #2013,
#    2014,
#    2015,
   2016,
   2017,
   2018,
])]

### Define `x's` and `y's`

In [ ]:
x_all = model_year.drop([
    'district', 
    'year', 
    'yield_mt',
    "crop_perc"
], axis = 1)

# y_all = features_summary.yield_mt
y_all = np.log10(model_year.yield_mt.to_numpy() + 1)

### Split into train and test sets

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
    x_all, y_all, test_size=0.2, random_state=0
)

In [ ]:
print("Total N: ", len(x_all), "\n", 
      "Train N: ", len(x_train), "\n",
      "Test  N: ", len(x_test), sep = "")

### Train model

In [ ]:
ridge_cv_random = RidgeCV(cv=5, alphas=np.logspace(-8, 8, base=10, num=17))
ridge_cv_random.fit(x_train, y_train)

### Validation set $R^2$ performance

In [ ]:
print(f"Validation R2 performance {ridge_cv_random.best_score_:0.2f}")

### Train set

In [ ]:
y_pred = np.maximum(ridge_cv_random.predict(x_train), 0)

plt.figure()
plt.scatter(y_pred, y_train, alpha=1, s=4)
plt.xlabel("Predicted", fontsize=15, x = .3)
plt.ylabel("Ground Truth", fontsize=15)
plt.suptitle(r"$\log_{10}(1 + Crop Yield)$", fontsize=20, y=1.02)
plt.title((f"Model applied to train data n = {len(x_train)}, R$^2$ = {(r2_score(y_train, y_pred)):0.2f}"),
          fontsize=12, y=1.01)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

m, b = np.polyfit(y_pred, y_train, 1)
plt.plot(y_pred, m * y_pred + b, color="black")
plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)

# plt.savefig(f'images/{feature_file_name}_train_data.jpg', dpi=300)
plt.show()
plt.close()

In [ ]:
print(f"Training set R^2 = {r2_score(y_train, y_pred):0.2f}")
print(f"Training set pearsons R = {pearsonr(y_pred, y_train)[0]:0.2f}") 

### Test set

In [ ]:
y_pred = np.maximum(ridge_cv_random.predict(x_test), 0)

plt.figure()
plt.scatter(y_pred, y_test, alpha=1, s=4)
plt.xlabel("Predicted", fontsize=15)
plt.ylabel("Ground Truth", fontsize=15)
plt.suptitle(r"$\log_{10}(1 + Crop Yield)$", fontsize=20, y=1.02)
plt.title(f"Model applied to test data n = {len(x_test)}, R$^2$ = {(r2_score(y_test, y_pred)):0.2f}",
          fontsize=12, y=1)

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)

m, b = np.polyfit(y_pred, y_test, 1)
plt.plot(y_pred, m * y_pred + b, color="black")
plt.gca().spines.right.set_visible(False)
plt.gca().spines.top.set_visible(False)

# plt.savefig(f'images/{feature_file_name}_test_data.jpg', dpi=300)
plt.show()
plt.close()

In [ ]:
print(f"Testing set R^2 = {r2_score(y_test, y_pred):0.2f}")
print(f"Testing set pearsons R = {pearsonr(y_pred, y_test)[0]:0.2f}")

### Plot the fitted features

In [ ]:
pred_features = features_copy.copy()
pred_features = pred_features.join(weights.drop('geometry', axis = 1).set_index(['lon', 'lat']), on = ['lon', 'lat'])

In [ ]:
pred_features = geopandas.GeoDataFrame(pred_features)
x_all = pred_features.drop([
    'year', 
    'lon',
    'lat',
    'crop_perc'
], axis = 1)
pred_features['fit'] = np.maximum(ridge_cv_random.predict(x_all), 0)

In [ ]:
pred_features['fit'].mask(pred_features['crop_perc']==0, 0, inplace=True)
# pred_features.loc[pred_features["crop_perc"] == 0, "fit"] = 0   ### Does same thing but differently

In [ ]:
# pred_features = pred_features[pred_features.crop_perc > 0].reset_index(drop = True)

In [ ]:
pred_features['fit'].mask(pred_features['fit'] > 4, 0, inplace=True)

In [ ]:
pred_features = geopandas.GeoDataFrame(
    pred_features, 
    geometry = geopandas.points_from_xy(x = pred_features.lon, y = pred_features.lat), 
    crs='EPSG:4326'
)

In [ ]:
plot_features = pred_features[pred_features.year == 2017]
# plot_features

In [ ]:
plot_features.plot(figsize = (10,10),
                   marker='H',
                   legend = True,
                   markersize = marker_sz,
#                    alpha = .9,
                   column = 'fit')

In [ ]:
# true_yield_2018 = residual_gdf[residual_gdf.year == 2018]['log_yield']
# residual_2018 = residual_gdf[residual_gdf.year == 2018]['residual']

# plt.figure()

# p = sns.jointplot(x=true_yield_2018, y = residual_2018).set_axis_labels("True Yield", "Residuals")
# # p.axhline(0)
# plt.suptitle("Plot of Yield vs Residuals 2018",
#           fontsize=12, y=1)

# plt.xticks(fontsize=14)
# plt.yticks(fontsize=14)

# plt.show()
# plt.close()

## Yield and Residual Plots

In [ ]:
x_all = features_summary.drop([
    'district', 
    'year', 
    'yield_mt',
    'crop_perc'
], axis = 1)

residual_df = pd.DataFrame()

residual_df["yield_mt"] = features_summary.yield_mt.to_numpy()
residual_df["log_yield"] = np.log10(features_summary.yield_mt.to_numpy() + 1)
residual_df["prediction"] = np.maximum(ridge_cv_random.predict(x_all), 0)
residual_df["residual"] = residual_df["log_yield"] - residual_df["prediction"]
#residual_df["demean_yield"] = residual_df["log_yield"] - residual_df["log_yield"].mean()
#residual_df["demean_prediction"] = residual_df["prediction"] - residual_df["prediction"].mean()
residual_df["year"] = features_summary.year
residual_df["district"] = features_summary.district
residual_df = residual_df.join(country_shp, how = "left", on = "district")

In [ ]:
residual_df

In [ ]:
#demean by location
residual_df["district_yield_mean"] = residual_df.groupby('district')['log_yield'].transform('mean')
residual_df["district_prediction_mean"] = residual_df.groupby('district')['prediction'].transform('mean')

residual_df["demean_yield"] = residual_df["log_yield"] - residual_df["district_yield_mean"]
residual_df["demean_prediction"] = residual_df["prediction"] - residual_df["district_prediction_mean"]
residual_df

In [ ]:
residual_gdf = geopandas.GeoDataFrame(residual_df)
residual_gdf

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
plt.suptitle("Yield (MT)")
plt.ax1 = residual_gdf[residual_gdf.year == 2016].hist(ax = ax1, column = "yield_mt", legend = True, bins = 20)
ax1.set_title("2016 Yield (MT)")
plt.ax2 = residual_gdf[residual_gdf.year == 2017].hist(ax = ax2, column = "yield_mt", legend = True, bins = 20) 
ax2.set_title("2017 Yield (MT)")
plt.ax3 = residual_gdf[residual_gdf.year == 2018].hist(ax = ax3, column = "yield_mt", legend = True, bins = 20)
ax3.set_title("2018 Yield (MT)")

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
plt.suptitle(r"$\log_{10}(1 + Crop Yield)$")
plt.ax1 = residual_gdf[residual_gdf.year == 2016].hist(ax = ax1, column = "log_yield", legend = True, bins = 20)
ax1.set_title(r"2016 $\log_{10}(1 + Crop Yield)$")
plt.ax2 = residual_gdf[residual_gdf.year == 2017].hist(ax = ax2, column = "log_yield", legend = True, bins = 20) 
ax2.set_title(r"2017 $\log_{10}(1 + Crop Yield)$")
plt.ax3 = residual_gdf[residual_gdf.year == 2018].hist(ax = ax3, column = "log_yield", legend = True, bins = 20)
ax3.set_title(r"2018 $\log_{10}(1 + Crop Yield)$")

In [ ]:
true_yield_2016 = residual_gdf[residual_gdf.year == 2016]['log_yield']
residual_2016 = residual_gdf[residual_gdf.year == 2016]['residual']

true_yield_2017 = residual_gdf[residual_gdf.year == 2017]['log_yield']
residual_2017 = residual_gdf[residual_gdf.year == 2017]['residual']

true_yield_2018 = residual_gdf[residual_gdf.year == 2018]['log_yield']
residual_2018 = residual_gdf[residual_gdf.year == 2018]['residual']

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5), sharey=True)
fig.suptitle('Residuals vs log10(Yield +1)')

sns.scatterplot(ax=ax1, x=true_yield_2016, y = residual_2016)
ax1.set_title(r"Plot of $\log_{10}(1 + Crop Yield)$ vs Residuals 2016")

sns.scatterplot(ax=ax2, x=true_yield_2017, y = residual_2017)
ax2.set_title(r"Plot of $\log_{10}(1 + Crop Yield)$ vs Residuals 2017")

sns.scatterplot(ax=ax3, x=true_yield_2018, y = residual_2018)
ax3.set_title(r"Plot of $\log_{10}(1 + Crop Yield)$ vs Residuals 2018")

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
ax1 = (residual_gdf[residual_gdf.year == 2016]
       .plot(ax = ax1, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
       .set_title("2016 Residuals"))
ax2 = (residual_gdf[residual_gdf.year == 2017]
       .plot(ax = ax2, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
       .set_title("2017 Residuals"))
ax3 = (residual_gdf[residual_gdf.year == 2018]
       .plot(ax = ax3, column = "residual", legend = True, norm=colors.Normalize(vmin= -0.4, vmax=0.4), cmap = "BrBG")
       .set_title("2018 Residuals"))

caption = "A positive value is an underestimated prediction (the prediction is lower than the actual yield), a negative value is an over estimated prediction"
plt.figtext(0.5, 0.01, caption, wrap=True, horizontalalignment='center', fontsize=12)

In [ ]:
fig, (ax1,ax2,ax3) = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
plt.suptitle("Residuals")
plt.ax1 = residual_gdf[residual_gdf.year == 2016].hist(ax = ax1, column = "residual", legend = True)
ax1.set_title("2016 Residuals")
plt.ax2 = residual_gdf[residual_gdf.year == 2017].hist(ax = ax2, column = "residual", legend = True) 
ax2.set_title("2017 Residuals")
plt.ax3 = residual_gdf[residual_gdf.year == 2018].hist(ax = ax3, column = "residual", legend = True)
ax3.set_title("2018 Residuals")

In [ ]:
residual_gdf.residual.min()

In [ ]:
residual_gdf.residual.max()

# Difference from the Mean

In [ ]:
residual_gdf_2016 = residual_gdf[residual_gdf.year == 2016]
residual_gdf_2017 = residual_gdf[residual_gdf.year == 2017]
residual_gdf_2018 = residual_gdf[residual_gdf.year == 2018]

In [ ]:
x1 = residual_gdf_2016["demean_yield"]
y1 = residual_gdf_2016["demean_prediction"]

x2 = residual_gdf_2017["demean_yield"]
y2 = residual_gdf_2017["demean_prediction"]

x3 = residual_gdf_2018["demean_yield"]
y3 = residual_gdf_2018["demean_prediction"]

x4 = residual_gdf["demean_yield"]
y4 = residual_gdf["demean_prediction"]



plt.scatter(x4, y4)
plt.title("Demeaned truth and predictions by district")
plt.xlabel('Difference from Yield Mean')
plt.ylabel('Difference from Predictions Mean')



# plt.subplots(figsize = (20, 5))
# plt.suptitle("Difference from the Mean Yield and Prediction")
# plt.tight_layout()

# plt.subplot(1, 4, 1)
# plt.scatter(x1, y1)
# plt.xlabel('Difference from Yield Mean')
# plt.ylabel('Difference from Predictions Mean')

# plt.subplot(1, 4, 2)
# plt.scatter(x2, y2)
# plt.title("2017")
# plt.xlabel('Difference from Yield Mean')
# plt.ylabel('Difference from Predictions Mean')

# plt.subplot(1, 4, 3)
# plt.scatter(x3, y3)
# plt.title("2018")
# plt.xlabel('Difference from Yield Mean')
# plt.ylabel('Difference from Predictions Mean')

# plt.subplot(1, 4, 4)
# plt.scatter(x4, y4)
# plt.title("All Years")
# plt.xlabel('Difference from Yield Mean')
# plt.ylabel('Difference from Predictions Mean')

In [ ]:
year_end

In [ ]:
for yr in range(year_start+1, year_end+1):
    r_squared = r2_score(residual_gdf[residual_gdf.year == yr]["demean_yield"], residual_gdf[residual_gdf.year == yr]["demean_prediction"])
    print(yr, "R^2: ", r_squared)
    pearson_r = pearsonr(residual_gdf[residual_gdf.year == yr]["demean_yield"], residual_gdf[residual_gdf.year == yr]["demean_prediction"])
    print(yr, "Pearson's: ", pearson_r[0])
    print("")